In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import keras
from models import model_v1

2024-02-25 18:55:16.159106: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-25 18:55:16.201640: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 18:55:16.201664: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 18:55:16.202990: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-25 18:55:16.209493: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# dataset
from pathlib import Path
from data_generator import DataGenerator

# training set data generator
root = "/home/ji/Dropbox/Robotics/CMSC733/Project1"
train_path = Path(root) / "Phase2" / "Data" / "Train"
train_gen = DataGenerator(train_path,mode="supervised")

# tensorflow dataset
im_shape = (train_gen.crop_size,train_gen.crop_size,3)
output_signature = ((tf.TensorSpec(shape=im_shape,dtype=tf.float32),
                     tf.TensorSpec(shape=im_shape,dtype=tf.float32)),
                     tf.TensorSpec(shape = (8,),dtype=tf.float32),
                    )
train_ds = tf.data.Dataset.from_generator(train_gen,
                                          output_signature=output_signature)

def config_ds(ds):
    AUTOTUNE = tf.data.AUTOTUNE
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    ds = ds.batch(8)
    return ds

train_ds = config_ds(train_ds)

2024-02-25 18:55:20.870824: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-02-25 18:55:20.870844: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: ji-GE76-Raider-11UE
2024-02-25 18:55:20.870849: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: ji-GE76-Raider-11UE
2024-02-25 18:55:20.870938: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 550.54.14
2024-02-25 18:55:20.870949: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  550.54.14  Release Build  (dvs-builder@U16-A24-2-2)  Thu Feb 22 01:44:50 UTC 2024
GCC version:  gcc version 9.4.0 (Ubuntu 9.4.0-1ubuntu1~20.04.2

In [5]:
# set up model
model = model_v1(verbose=True)

def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true-y_pred))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=custom_loss)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 tf.__operators__.getitem_2  (None, 128, 128, 3)          0         ['input_5[0][0]']             
  (SlicingOpLambda)                                                                               
                                                                                                  
 tf.__operators__.getitem_3  (None, 128, 128, 3)          0         ['input_6[0][0]']       

In [6]:
# reduce learning rate on plateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
                                                factor=0.2,
                                                patience=5,
                                                min_lr=1e-6,
                                                verbose=1,
                                                cooldown=5)

# save check points
checkpoint_filepath = './checkpoint'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='loss',
    mode='min',
    save_best_only=True)

history = model.fit(train_ds,epochs=150,verbose=True,steps_per_epoch=150,
                    callbacks=[reduce_lr,checkpoint_callback])

Epoch 1/150
150/150 [==============================] - ETA: 0s - loss: 8.4034INFO:tensorflow:Assets written to: ./checkpoint/assets


INFO:tensorflow:Assets written to: ./checkpoint/assets


150/150 [==============================] - 63s 411ms/step - loss: 8.4034 - lr: 0.0010
Epoch 2/150
150/150 [==============================] - ETA: 0s - loss: 8.3020INFO:tensorflow:Assets written to: ./checkpoint/assets


INFO:tensorflow:Assets written to: ./checkpoint/assets


150/150 [==============================] - 62s 416ms/step - loss: 8.3020 - lr: 0.0010
Epoch 3/150
150/150 [==============================] - ETA: 0s - loss: 7.8253INFO:tensorflow:Assets written to: ./checkpoint/assets


...

INFO:tensorflow:Assets written to: ./checkpoint/assets


150/150 [==============================] - 64s 425ms/step - loss: 7.4190 - lr: 0.0010
Epoch 6/150
150/150 [==============================] - ETA: 0s - loss: 7.2999INFO:tensorflow:Assets written to: ./checkpoint/assets


INFO:tensorflow:Assets written to: ./checkpoint/assets


150/150 [==============================] - 64s 427ms/step - loss: 7.2999 - lr: 0.0010
Epoch 7/150
150/150 [==============================] - 62s 415ms/step - loss: 7.3019 - lr: 0.0010
Epoch 8/150
 12/150 [=>............................] - ETA: 59s - loss: 7.4702

KeyboardInterrupt: 